# Moderation Analysis: Individual Differences in Response to AI Uncertainty

## Research Question

While our main hypothesis testing showed no significant main effect of uncertainty communication on trust in AI systems, **moderation analysis** asks a more nuanced question:

> **For whom or under what conditions does uncertainty communication affect trust?**

Even when an overall effect is null, different subgroups may respond differently to the intervention:
- High-tech-affinity individuals might trust the AI *more* when shown uncertainty (transparency valued)
- Low-tech-affinity individuals might trust it *less* (uncertainty perceived as incompetence)
- These opposing effects could cancel out, producing the observed null main effect

## Statistical Framework: Moderation via Interaction Terms

### Conceptual Definition

A **moderator** (M) is a variable that affects the direction and/or strength of the relationship between an independent variable (X) and a dependent variable (Y).

### Mathematical Model

For a continuous moderator:

$$Y = \beta_0 + \beta_1 X + \beta_2 M + \beta_3 (X \times M) + \epsilon$$

Where:
- $Y$ = outcome (TiA subscale scores)
- $X$ = treatment (stimulus_group: uncertainty vs control)
- $M$ = moderator (e.g., ATI score)
- $\beta_1$ = main effect of treatment (conditional effect when M = 0)
- $\beta_2$ = main effect of moderator
- $\beta_3$ = **interaction effect** (the moderation effect)
- $\epsilon$ = error term

### Interpretation of Interaction Term

The interaction coefficient ($\beta_3$) tells us:
- Whether the effect of uncertainty communication **changes** as the moderator changes
- A significant $\beta_3$ indicates moderation is present
- The sign indicates the direction of moderation

**Example**: If ATI moderates the uncertainty→trust relationship:
- Positive $\beta_3$: Higher ATI → uncertainty communication increases trust more
- Negative $\beta_3$: Higher ATI → uncertainty communication increases trust less (or decreases it more)

## Best Practices Implemented

### 1. **Variable Centering**
Continuous predictors will be **mean-centered** before computing interaction terms:
- Makes $\beta_1$ interpretable as the effect at the average moderator level
- Reduces multicollinearity between interaction term and main effects
- Doesn't change the interaction coefficient itself

### 2. **Effect Coding for Categorical Variables**
Using effect coding (-0.5, 0.5) instead of dummy coding (0, 1):
- Main effects represent differences from the grand mean
- More symmetric interpretation
- Better for interactions with continuous variables

### 3. **Multiple Comparison Awareness**
- Theoretically motivated moderators: Standard α = 0.05
- Exploratory analyses: Consider adjusted α or interpret with caution
- Focus on effect sizes and confidence intervals, not just p-values

### 4. **Model Assumptions**
While we won't exhaustively check here, be aware of:
- Linearity of relationships
- Homoscedasticity (constant variance)
- Normality of residuals
- Independence of observations

## Analysis Structure

1. **Data preparation**: Load data, center variables, create interaction terms
2. **Theoretically motivated moderators**:
   - Affinity for Technology Interaction (ATI)
   - Healthcare trust - competence (healthcare_trust_c)
   - Healthcare trust - values (healthcare_trust_v)
   - Demographics: gender, age, education, medical background
3. **Exploratory analysis**: Test remaining potential moderators
4. **Summary**: Synthesize findings

## Setup and Data Loading

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests

from scripts.stats import test_moderation, print_moderation_results

import warnings
warnings.filterwarnings('ignore')

# Set display options for better readability
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', None)

# define variable names
tia_scales = ['tia_f', 'tia_pro', 'tia_rc', 'tia_t', 'tia_up']
hcsds_scales = ['hcsds_c', 'hcsds_v']
ati_scales = ['ati']
manip_check_scales = []
for i in range(4):
    manip_check_scales.append(f'manip_check1_{i+1}')

scale_titles = {
    'ati': 'Affinity for Technology Interaction',
    'hcsds_c': 'Healthcare Trust - Competence',
    'hcsds_v': 'Healthcare Trust - Values',
    'tia_rc': 'TiA - Reliability/Confidence',
    'tia_up': 'TiA - Understanding/Predictability',
    'tia_f': 'TiA - Familiarity',
    'tia_pro': 'TiA - Propensity to Trust',
    'tia_t': 'TiA - Trust in Automation'
}

In [2]:
# Load data
data = pd.read_csv('../data/data_scales.csv')

print(f"Total sample size: {len(data)}")
print(f"\nGroup distribution:")
print(data['stimulus_group'].value_counts())
print(f"\nTiA outcome variables:")
print(tia_scales)

Total sample size: 255

Group distribution:
stimulus_group
1    129
0    126
Name: count, dtype: int64

TiA outcome variables:
['tia_f', 'tia_pro', 'tia_rc', 'tia_t', 'tia_up']


## Data Preparation

### Variable Centering and Coding

We'll prepare our variables for moderation analysis:

1. **Effect code the treatment**: stimulus_group as -0.5 (control) and 0.5 (uncertainty)
2. **Mean-center continuous moderators**: ATI, healthcare trust subscales, age
3. **Effect code categorical variables**: gender, education, medical background

**Why mean-center?**
- Interpretation: Main effect of X represents the effect when M is at its mean
- Statistical: Reduces multicollinearity in interaction models
- Practical: Coefficients are more meaningful

**Note on centering**: We use grand-mean centering (subtract overall mean) rather than group-mean centering, appropriate for testing moderation.

In [3]:
# Create working copy
df = data.copy()

# 1. Effect code treatment variable
# control = -0.5, uncertainty = 0.5
df['group_effect'] = df['stimulus_group'] - 0.5

# 2. Mean-center continuous moderators
continuous_moderators = hcsds_scales + ati_scales + ['age']

for var in continuous_moderators:
    df[f'{var}_c'] = df[var] - df[var].mean()
    print(f"{var}: Mean = {df[var].mean():.3f}, SD = {df[var].std():.3f}")

# 3. Effect code categorical variables (will do this per-analysis as needed)
# For now, just check their distributions
print("\nCategorical variable distributions:")
for var in ['gender', 'education', 'Q19']:
    print(f"\n{var}:")
    print(df[var].value_counts())

hcsds_c: Mean = 3.535, SD = 0.689
hcsds_v: Mean = 3.155, SD = 0.679
ati: Mean = 2.888, SD = 0.805
age: Mean = 27.345, SD = 13.604

Categorical variable distributions:

gender:
gender
2    157
1     93
3      5
Name: count, dtype: int64

education:
education
6    131
7     60
3     31
4     11
8      9
2      7
5      6
Name: count, dtype: int64

Q19:
Q19
3    88
4    68
2    49
1    27
5    23
Name: count, dtype: int64


## Theoretically Motivated Moderators

We test moderation for variables with theoretical justification:

### 1. Affinity for Technology Interaction (ATI)

**Theoretical Rationale**: Individuals with higher technology affinity may:
- Better understand the meaning and implications of AI uncertainty
- Value transparency in AI systems more highly
- Have more realistic mental models of AI capabilities

**Hypothesis**: ATI moderates the uncertainty→trust relationship (direction exploratory)

### 2. Healthcare System Trust

**Theoretical Rationale**:
- Trust in healthcare systems may transfer to trust in medical AI
- Those who distrust healthcare competence may be more skeptical of AI uncertainty
- Alignment of values affects receptiveness to transparency

**Hypothesis**: Both competence and values dimensions may moderate the effect

### 3. Demographics

**Theoretical Rationale**:
- **Age**: Older adults may have different AI literacy and trust baselines
- **Gender**: Literature shows gender differences in technology trust
- **Education**: Higher education may correlate with statistical literacy
- **AI Experience** (Q19): Self-reported AI-experience might be predictor of trust calibration.

### Continuous Moderators: ATI and Healthcare Trust

We test each continuous moderator against all TiA subscales:
- TiA_rc: Reliability/Competence
- TiA_up: Understanding/Predictability
- TiA_f: Familiarity
- TiA_pro: Propensity to Trust
- TiA_t: General Trust in Automation

In [4]:
# Store results for summary
all_results = []

In [5]:
print("="*80)
print("MODERATOR: AFFINITY FOR TECHNOLOGY INTERACTION (ATI)")
print("="*80)

for outcome in tia_scales:
    results, model = test_moderation(df, outcome, 'ati', 'ati_c', categorical=False)
    print_moderation_results(results)
    all_results.append(results)

MODERATOR: AFFINITY FOR TECHNOLOGY INTERACTION (ATI)

Outcome: tia_f | Moderator: ati
N = 255, R² = 0.0777, Adj. R² = 0.0667

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       0.1650     0.1112     [-0.054,  0.384]          0.1390    
ati                       0.3037     0.0694                               <.001     
Interaction               0.0974     0.1387     [-0.176,  0.371]          0.4833    

Interpretation:          
  No significant moderation effect detected.


Outcome: tia_pro | Moderator: ati
N = 255, R² = 0.0027, Adj. R² = -0.0092

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       -0.0324    0.0831     [-0.196,  0.131]          0.6971    
ati                       -0.0370    0.0518          

In [6]:
for moderator in hcsds_scales:
    print("="*80)
    print(f"MODERATOR: {scale_titles[moderator]}")
    print("="*80)

    for outcome in tia_scales:
        results, model = test_moderation(df, outcome, scale_titles[moderator],
                                         moderator, categorical=False)
        print_moderation_results(results)
        all_results.append(results)

MODERATOR: Healthcare Trust - Competence

Outcome: tia_f | Moderator: Healthcare Trust - Competence
N = 255, R² = 0.0109, Adj. R² = -0.0010

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       0.4824     0.6114     [-0.722,  1.687]          0.4308    
Healthcare Trust - Competence -0.0665    0.0848                               0.4337    
Interaction               -0.0960    0.1695     [-0.430,  0.238]          0.5715    

Interpretation:          
  No significant moderation effect detected.


Outcome: tia_pro | Moderator: Healthcare Trust - Competence
N = 255, R² = 0.0193, Adj. R² = 0.0076

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       0.4653     0.4375     [-0.396,  1.327]          0.2885    
Healthcar

### Demographic Moderators

For categorical variables with 2 levels (gender), we use effect coding.

For multi-level categorical variables (education, AI experience), we need to consider:
- Whether to treat as categorical (multiple dummy variables)
- Whether there's a meaningful ordering (ordinal)
- Statistical power with smaller cell sizes

For age, we treat as continuous (already centered above).

In [7]:
print("="*80)
print("MODERATOR: AGE")
print("="*80)

for outcome in tia_scales:
    results, model = test_moderation(df, outcome, 'Age', 'age_c', categorical=False)
    print_moderation_results(results)
    all_results.append(results)

MODERATOR: AGE

Outcome: tia_f | Moderator: Age
N = 255, R² = 0.0327, Adj. R² = 0.0211

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       0.1404     0.1139     [-0.084,  0.365]          0.2188    
Age                       0.0109     0.0042                               0.0100    
Interaction               0.0002     0.0084     [-0.016,  0.017]          0.9829    

Interpretation:          
  No significant moderation effect detected.


Outcome: tia_pro | Moderator: Age
N = 255, R² = 0.0383, Adj. R² = 0.0268

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       -0.0375    0.0816     [-0.198,  0.123]          0.6459    
Age                       0.0094     0.0030                               0.0020    
Interac

In [8]:
# Check gender distribution to decide on coding
print("Gender distribution:")
print(df['gender'].value_counts())
print("\nNote: Effect coding requires adequate sample sizes in both groups.")

Gender distribution:
gender
2    157
1     93
3      5
Name: count, dtype: int64

Note: Effect coding requires adequate sample sizes in both groups.


In [9]:
# Effect code gender (assuming two main categories)
# Adjust this based on actual data structure
# Common approach: female = -0.5, male = 0.5 (or vice versa)
# If there are other categories with sufficient n, may need different approach

# For demonstration, creating effect-coded version
# You may need to adjust based on actual gender categories in data
gender_mapping = df['gender'].value_counts().index[:2]  # Take two most common
print(f"Using gender categories: {gender_mapping.tolist()}")

# Filter to just these two categories for this analysis
df_gender = df[df['gender'].isin(gender_mapping)].copy()
df_gender['gender_effect'] = df_gender['gender'].map({gender_mapping[0]: -0.5, gender_mapping[1]: 0.5})

print(f"\nSample size for gender analysis: {len(df_gender)}")
print(df_gender['gender_effect'].value_counts())

Using gender categories: [2, 1]

Sample size for gender analysis: 250
gender_effect
-0.5    157
 0.5     93
Name: count, dtype: int64


In [10]:
print("="*80)
print("MODERATOR: GENDER")
print("="*80)

for outcome in tia_scales:
    results, model = test_moderation(df_gender, outcome, 'Gender', 'gender_effect', categorical=True)
    print_moderation_results(results)
    # Note: Not adding to all_results since different sample size
    all_results.append(results)

MODERATOR: GENDER

Outcome: tia_f | Moderator: Gender
N = 250, R² = 0.0172, Adj. R² = 0.0052

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       0.1775     0.1199     [-0.059,  0.414]          0.1402    
Gender                    0.1642     0.1199                               0.1722    
Interaction               0.1673     0.2398     [-0.305,  0.640]          0.4862    

Interpretation:          
  No significant moderation effect detected.


Outcome: tia_pro | Moderator: Gender
N = 250, R² = 0.0149, Adj. R² = 0.0029

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       0.0232     0.0858     [-0.146,  0.192]          0.7869    
Gender                    0.1282     0.0858                               0.1365   

In [11]:
# Education: Treat as ordinal (meaningful hierarchy)
df['education_c'] = df['education'] - df['education'].mean()

print("="*80)
print("MODERATOR: EDUCATION (ORDINAL)")
print("="*80)


for outcome in tia_scales:
    results, model = test_moderation(df, outcome, 'Education',
                                     'education_c', categorical=False)
    print_moderation_results(results)
    all_results.append(results)

MODERATOR: EDUCATION (ORDINAL)

Outcome: tia_f | Moderator: Education
N = 255, R² = 0.0156, Adj. R² = 0.0038

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       0.1530     0.1149     [-0.073,  0.379]          0.1842    
Education                 0.0542     0.0405                               0.1822    
Interaction               0.0465     0.0810     [-0.113,  0.206]          0.5671    

Interpretation:          
  No significant moderation effect detected.


Outcome: tia_pro | Moderator: Education
N = 255, R² = 0.0115, Adj. R² = -0.0003

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       -0.0334    0.0827     [-0.196,  0.130]          0.6866    
Education                 -0.0374    0.0292                    

In [12]:
# AI Experience (Q19): Treats ordinal (meaningful hierarchy)
df['Q19_c'] = df['Q19'] - df['Q19'].mean()

print("="*80)
print("MODERATOR: AI Experience (self-reported) (ORDINAL)")
print("="*80)


for outcome in tia_scales:
    results, model = test_moderation(df, outcome, 'AI Experience',
                                     'Q19_c', categorical=False)
    print_moderation_results(results)
    all_results.append(results)

MODERATOR: AI Experience (self-reported) (ORDINAL)

Outcome: tia_f | Moderator: AI Experience
N = 255, R² = 0.0097, Adj. R² = -0.0021

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       0.1436     0.1154     [-0.084,  0.371]          0.2143    
AI Experience             0.0376     0.0519                               0.4690    
Interaction               -0.0584    0.1038     [-0.263,  0.146]          0.5740    

Interpretation:          
  No significant moderation effect detected.


Outcome: tia_pro | Moderator: AI Experience
N = 255, R² = 0.0158, Adj. R² = 0.0040

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       -0.0332    0.0827     [-0.196,  0.130]          0.6880    
AI Experience             0.0223   

## Summary of Theoretically Motivated Moderators

Let's create a summary table of all moderation effects tested.

In [13]:
# Create summary DataFrame
summary_df = pd.DataFrame(all_results)

# Select key columns for display
summary_display = summary_df[[
    'outcome', 'moderator', 'n',
    'b_interaction', 'se_interaction',
    'ci_interaction_lower', 'ci_interaction_upper',
    'p_interaction'
]].copy()

# Add significance markers
summary_display['sig'] = summary_display['p_interaction'].apply(
    lambda p: '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns'
)

# Format for display
summary_display['β (interaction)'] = summary_display.apply(
    lambda row: f"{row['b_interaction']:.3f} ({row['se_interaction']:.3f})", axis=1
)
summary_display['95% CI'] = summary_display.apply(
    lambda row: f"[{row['ci_interaction_lower']:.3f}, {row['ci_interaction_upper']:.3f}]", axis=1
)
summary_display['p'] = summary_display['p_interaction'].apply(
    lambda p: f"{p:.4f}" if p >= 0.001 else "<.001"
)

# Display summary
print("\n" + "="*100)
print("SUMMARY: THEORETICALLY MOTIVATED MODERATION ANALYSES")
print("="*100)
print("\nInteraction Effects (β and SE):\n")
display_cols = ['outcome', 'moderator', 'n', 'β (interaction)', '95% CI', 'p', 'sig']
print(summary_display[display_cols].to_string(index=False))

# Count significant effects
n_sig = (summary_display['p_interaction'] < 0.05).sum()
n_total = len(summary_display)
print(f"\n{'='*100}")
print(f"Significant moderation effects: {n_sig}/{n_total} ({n_sig/n_total*100:.1f}%)")
print(f"{'='*100}\n")


SUMMARY: THEORETICALLY MOTIVATED MODERATION ANALYSES

Interaction Effects (β and SE):

outcome                     moderator   n β (interaction)           95% CI      p sig
  tia_f                           ati 255   0.097 (0.139)  [-0.176, 0.371] 0.4833  ns
tia_pro                           ati 255  -0.027 (0.104)  [-0.231, 0.177] 0.7962  ns
 tia_rc                           ati 255   0.037 (0.087)  [-0.135, 0.209] 0.6743  ns
  tia_t                           ati 255   0.092 (0.124)  [-0.153, 0.337] 0.4600  ns
 tia_up                           ati 255  -0.041 (0.114)  [-0.266, 0.184] 0.7219  ns
  tia_f Healthcare Trust - Competence 255  -0.096 (0.170)  [-0.430, 0.238] 0.5715  ns
tia_pro Healthcare Trust - Competence 255  -0.137 (0.121)  [-0.376, 0.102] 0.2592  ns
 tia_rc Healthcare Trust - Competence 255  -0.073 (0.100)  [-0.270, 0.124] 0.4661  ns
  tia_t Healthcare Trust - Competence 255   0.159 (0.142)  [-0.120, 0.438] 0.2627  ns
 tia_up Healthcare Trust - Competence 255   0.014 (0

## Exploratory Moderation Analysis

### Rationale and Cautions

Beyond theoretically motivated moderators, we can explore whether other measured variables moderate the uncertainty→trust relationship.

**Important considerations:**

1. **Multiple comparisons**: Testing many moderators inflates Type I error rate
   - Consider Bonferroni or FDR correction
   - Interpret nominally significant findings cautiously
   - Treat as hypothesis-generating, not confirmatory

2. **Theoretical justification**: Post-hoc explanations for significant findings should be viewed skeptically

3. **Effect sizes**: Focus on magnitude of effects, not just p-values

### Potential Exploratory Moderators

Depending on what else is in your dataset:
- Time spent viewing stimulus (page_submit)
- Other demographic variables
- Any additional measured individual differences

**Note**: We'll list available variables and you can select which to explore.

In [14]:
# Select exploratory moderators to test
# Modify this list based on what's theoretically interesting
exploratory_moderators = manip_check_scales + ['page_submit']

print(f"Testing {len(exploratory_moderators)} exploratory moderators:")
print(exploratory_moderators)

if len(exploratory_moderators) == 0:
    print("\nNo additional moderators selected for exploratory analysis.")
    print("You can manually add variables to 'exploratory_moderators' list above.")

Testing 5 exploratory moderators:
['manip_check1_1', 'manip_check1_2', 'manip_check1_3', 'manip_check1_4', 'page_submit']


In [15]:
# Run exploratory moderation analyses
exploratory_results = []

if len(exploratory_moderators) > 0:
    print("="*80)
    print("EXPLORATORY MODERATION ANALYSES")
    print("="*80)
    print(f"Testing {len(exploratory_moderators)} moderators × {len(tia_scales)} outcomes")
    print(f"Total tests: {len(exploratory_moderators) * len(tia_scales)}")
    print(f"Bonferroni-corrected α: {0.05 / (len(exploratory_moderators) * len(tia_scales)):.6f}")
    print("="*80)

    for moderator in exploratory_moderators:
        # Center the moderator
        moderator_c = f"{moderator}_c_exp"
        df[moderator_c] = df[moderator] - df[moderator].mean()

        print(f"\n{'-'*80}")
        print(f"MODERATOR: {moderator}")
        print(f"{'-'*80}")

        for outcome in tia_scales:
            results, model = test_moderation(df, outcome, moderator, moderator_c, categorical=False)
            print_moderation_results(results)
            exploratory_results.append(results)

    # Apply multiple comparison correction
    p_values = [r['p_interaction'] for r in exploratory_results]

    # Bonferroni correction
    reject_bonf, pvals_bonf, _, _ = multipletests(p_values, alpha=0.05, method='bonferroni')

    # FDR correction (less conservative)
    reject_fdr, pvals_fdr, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

    # Add to results
    for i, results in enumerate(exploratory_results):
        results['p_bonferroni'] = pvals_bonf[i]
        results['sig_bonferroni'] = reject_bonf[i]
        results['p_fdr'] = pvals_fdr[i]
        results['sig_fdr'] = reject_fdr[i]

    # Summary
    exp_summary = pd.DataFrame(exploratory_results)

    print("\n" + "="*100)
    print("EXPLORATORY ANALYSIS SUMMARY (with multiple comparison corrections)")
    print("="*100)

    exp_summary['β (SE)'] = exp_summary.apply(
        lambda row: f"{row['b_interaction']:.3f} ({row['se_interaction']:.3f})", axis=1
    )

    display_exp = exp_summary[['outcome', 'moderator', 'β (SE)', 'p_interaction',
                                'p_bonferroni', 'sig_bonferroni', 'p_fdr', 'sig_fdr']].copy()

    print("\n" + display_exp.to_string(index=False))

    n_sig_uncorr = (exp_summary['p_interaction'] < 0.05).sum()
    n_sig_bonf = exp_summary['sig_bonferroni'].sum()
    n_sig_fdr = exp_summary['sig_fdr'].sum()

    print(f"\n{'='*100}")
    print(f"Significant effects (uncorrected α=.05): {n_sig_uncorr}/{len(exploratory_results)}")
    print(f"Significant effects (Bonferroni): {n_sig_bonf}/{len(exploratory_results)}")
    print(f"Significant effects (FDR): {n_sig_fdr}/{len(exploratory_results)}")
    print(f"{'='*100}\n")

EXPLORATORY MODERATION ANALYSES
Testing 5 moderators × 5 outcomes
Total tests: 25
Bonferroni-corrected α: 0.002000

--------------------------------------------------------------------------------
MODERATOR: manip_check1_1
--------------------------------------------------------------------------------

Outcome: tia_f | Moderator: manip_check1_1
N = 255, R² = 0.0134, Adj. R² = 0.0016

Coefficient               β          SE         95% CI                    p         
--------------------------------------------------------------------------------
Group (conditional)       0.1418     0.1151     [-0.085,  0.368]          0.2191    
manip_check1_1            -0.0643    0.0493                               0.1929    
Interaction               -0.0322    0.0985     [-0.226,  0.162]          0.7441    

Interpretation:          
  No significant moderation effect detected.


Outcome: tia_pro | Moderator: manip_check1_1
N = 255, R² = 0.0023, Adj. R² = -0.0096

Coefficient               β    

## Overall Summary and Interpretation

### Key Findings

Review the results above to answer:

1. **Which moderators showed significant interactions?**
   - What is the pattern of moderation?
   - What is the effect size?

2. **Are effects consistent across TiA subscales?**
   - Do some trust components show moderation while others don't?
   - What might this tell us about the construct?

3. **Theoretical implications**
   - Do findings support existing theories?
   - What new questions emerge?

### Next Steps

1. **Visualization**: Plot interactions for significant moderators
   - Simple slopes analysis
   - Interaction plots at ±1 SD of moderator
   - Johnson-Neyman regions of significance

2. **Model diagnostics**: Check assumptions for key models
   - Residual plots
   - Influence diagnostics
   - Homoscedasticity tests

3. **Robustness checks**:
   - Test with different moderator coding schemes
   - Examine outliers
   - Consider non-linear moderator effects

### Statistical Reporting

For each significant moderation, report:
- Interaction coefficient with SE and 95% CI
- p-value (exact if p > .001)
- R² or R² change for the full model
- Simple slopes at meaningful moderator values
- Visualization of the interaction

Example text:
> "ATI significantly moderated the effect of uncertainty communication on general trust,
> β = 0.15, SE = 0.06, 95% CI [0.03, 0.27], p = .013. Simple slopes analysis revealed
> that..."

In [16]:
# Export results for further analysis or visualization
summary_df.to_csv('../output/moderation_results_theoretical.csv', index=False)
print("Theoretically motivated moderation results saved to: data/moderation_results_theoretical.csv")

if len(exploratory_results) > 0:
    exp_summary.to_csv('../output/moderation_results_exploratory.csv', index=False)
    print("Exploratory moderation results saved to: data/moderation_results_exploratory.csv")

Theoretically motivated moderation results saved to: data/moderation_results_theoretical.csv
Exploratory moderation results saved to: data/moderation_results_exploratory.csv
